This notebook will prepare the training data
before running this, we first need to run the reslicing notebook to reslice everything to 5mm
then in this notebook
we first exclude everything that is below 28 or beyond 50 scans
then for each input scan, get all of the images and prepare a label for them
generate 1 numpy array for each slice in each scan

In [19]:
#read nii files
import os
import numpy as np
import nibabel as nib

import pandas as pd
from matplotlib import pyplot as plt
from skimage.transform import resize

In [20]:
label_file_name = 'summary_label.csv'
nifti_folder = 'D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles'
new_size = (224, 224)
#the following 2 parameters can not be 1 at the same time
normalize_img = 0
#if 1, normalize to 0~255
#if 0, don't normalize, use raw numbers
three_channels = 1
#if 1, create 3 channels instead of the original 1 channel
#if 0, use the original 1 channle
#for the 3 channels, we use the following window:
#brain window: (l = 40, w = 80), i.e. 0~80 (everything <0 is 0, everything >80 is 255)
#bone window: (l = 500, w = 3000), i.e. -1000~2000
#subdural window: (l = 175, w = 50), i.e. 150~200
#l is the center of the luminance and w is the width
channel_param = ((0, 80), (-1000, 2000), (150, 200))

#we will only include data that is within this range after reslicing to 5mm
max_num_slice = 50
min_num_slice = 28

In [21]:
def create_channel(img, channel_min, channel_max):
    #this function will create channeled data based on the original single values data
    
    img_channel = img.copy()
    img_channel[img_channel <= channel_min] = 0
    img_channel[img_channel >= channel_max] = channel_max
    img_channel = 255 * (img_channel-np.amin(img_channel))/(np.amax(img_channel) - np.amin(img_channel))
    return img_channel

    

In [22]:
#we need to count how many slices are in each nifti file, probably go with the smallest number
#then for each scan, get the middle slices with that number
#after preprocessing, we don't have scans from 120, 163, 336 and 491
scan_list = np.array([range(1, 120)])
scan_list = np.concatenate((scan_list, np.array([range(121, 163)])), axis=1)
scan_list = np.concatenate((scan_list, np.array([range(164, 336)])), axis=1)
scan_list = np.concatenate((scan_list, np.array([range(337, 491)])), axis=1)

num_slice = np.empty((scan_list.shape[1],))

j=0
for ss in scan_list[0]:
    #for each scan read in the nifti file
    i = ss
    scan_fn = nifti_folder + '/CQ500-CT-' + str(i) + '/CQ500-CT-' + str(i) + '_5mm.nii'
    img = nib.load(scan_fn)
    img_data = img.get_data()
    num_slice[j] = img_data.shape[2]
    j=j+1
    
num_slice = num_slice.astype(int)

scan_list=np.squeeze(scan_list)
#create a pandas dataframe with the index and num_slice
num_slice_df = pd.DataFrame({'scan_id':scan_list, 'num_slice':num_slice})

num_slice_df = num_slice_df[num_slice_df.num_slice <= max_num_slice]
num_slice_df = num_slice_df[num_slice_df.num_slice >= min_num_slice]

#num_slice_df has all of the information about number of slices in each scan

FileNotFoundError: No such file or no access: 'D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles/CQ500-CT-1/CQ500-CT-1_5mm.nii'

In [13]:
import os
import nibabel as nib

# Base directory where NIfTI files are stored
nifti_folder = 'D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles'

# Initialize a variable to store the number of slices (just as an example)
num_slice = {}

# Traverse through each directory from CQ500-CT-1 to CQ500-CT-490
for i in range(1, 491):  # Range goes from 1 to 490 inclusive
    folder_name = f'CQ500-CT-{i}'
    folder_path = os.path.join(nifti_folder, folder_name)
    
    # Check if the directory exists
    if os.path.isdir(folder_path):
        # Traverse all files in the directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.nii.gz'):  # Check if it's a NIfTI file
                scan_fn = os.path.join(folder_path, file_name)
                
                # Try loading the file
                try:
                    img = nib.load(scan_fn)
                    img_data = img.get_fdata()
                    num_slice[i] = img_data.shape[2]  # Store number of slices or perform other operations
                    print(f"Loaded {scan_fn}, number of slices: {img_data.shape[2]}")
                except Exception as e:
                    print(f"Error loading {scan_fn}: {e}")
    else:
        print(f"Directory does not exist: {folder_path}")


Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-1\CT_2.55mm_2.5_5mm_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-1\CT_2.55mm_2.5_5mm_0_2_Tilt_1.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-2\CT_5mm_5mm_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-2\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-3\CT_Plain_Plain_0_2.nii.gz, number of slices: 30
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-3\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 30
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-4\CT_5mm_5mm_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-4\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOu

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-45\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 28
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-46\CT_Plain_3mm_Plain_3mm_0_3.nii.gz, number of slices: 57
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-47\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 29
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-48\CT_Plain_Plain_0_2.nii.gz, number of slices: 34
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-49\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 29
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-50\CT_5mm_5mm_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-50\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-51

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-92\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-93\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 30
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-94\CT_Plain_3mm_Plain_3mm_0_2.nii.gz, number of slices: 52
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-95\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 30
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-96\CT_Plain_Plain_0_2.nii.gz, number of slices: 23
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-96\CT_Plain_Plain_0_2_Tilt_Eq_1.nii.gz, number of slices: 23
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-97\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 31
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/Nift

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-139\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-140\CT_Plain_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-140\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-141\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 31
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-142\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-143\CT_Plain_Plain_0_2.nii.gz, number of slices: 28
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-143\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 28
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-144\CT_PRE_CONTRAST_5MM_ST

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-181\CT_5mm_5mm_0_3_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-182\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 31
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-183\CT_Plain_Plain_0_2.nii.gz, number of slices: 44
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-184\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-184\CT_55mm_Plain_5_5mm_Plain_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-185\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-186\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFile

...

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-318\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 40
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-319\CT_5mm_5mm_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-319\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-320\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-321\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-322\CT_Plain_THIN_Plain_THIN_0_3.nii.gz, number of slices: 319
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-322\CT_Plain_THIN_Plain_THIN_0_3_Tilt_1.nii.gz, number of slices: 319
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-323\CT_Plain_Plain_0_2.nii.gz, 

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-361\CT_5mm_5mm_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-361\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-362\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-363\CT_Plain_Plain_0_2.nii.gz, number of slices: 28
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-363\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 28
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-364\CT_Plain_3mm_Plain_3mm_0_2.nii.gz, number of slices: 50
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-365\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-365\CT_55mm_Plain_5_5mm_Plain_0_2_Tilt_1.nii.gz, number 

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-401\CT_0.625mm-2_0.625mm_0_5.nii.gz, number of slices: 256
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-401\CT_0.625mm-2_0.625mm_0_5_Tilt_1.nii.gz, number of slices: 256
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-402\CT_5mm_5mm_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-402\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-403\CT_PRE_CONTRAST_5MM_STD_PRE_CONTRAST_5MM_STD_0_4.nii.gz, number of slices: 29
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-404\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-405\CT_55mm_Plain_5_5mm_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-406\CT_Plain_Plain_0

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-445\CT_Plain_THIN_Plain_THIN_0_3.nii.gz, number of slices: 307
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-446\CT_Plain_Plain_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-446\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-447\CT_BRAIN_PLAIN_BRAIN_PLAIN_0_2.nii.gz, number of slices: 32
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-448\CT_PLAIN_THIN_PLAIN_THIN_0_3.nii.gz, number of slices: 214
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-449\CT_0.625mm-2_0.625mm_0_3.nii.gz, number of slices: 256
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-449\CT_0.625mm-2_0.625mm_0_3_Tilt_1.nii.gz, number of slices: 256
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-450\CT_Plain_Plain_0_2.nii.gz,

Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-489\CT_5mm_5mm_0_2.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-489\CT_5mm_5mm_0_2_Tilt_1.nii.gz, number of slices: 36
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-490\CT_Plain_Plain_0_2.nii.gz, number of slices: 30
Loaded D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles\CQ500-CT-490\CT_Plain_Plain_0_2_Tilt_1.nii.gz, number of slices: 30


In [23]:
#now we will read in each scan, select min_num_slices from the middle of the scan, and
#generate a numpy file for each slice, with the corresponding label!

final_scan_list = num_slice_df['scan_id']

labels = pd.read_csv(label_file_name)

#use the scan name as index
labels.set_index('name', inplace=True)
        
for ss in final_scan_list:
    #for each scan read in the nifti file
    scan_fn = nifti_folder + '/CQ500-CT-' + str(ss) + '/CQ500-CT-' + str(ss) + '_5mm.nii'
    img = nib.load(scan_fn)
    img_data = img.get_data()
    
    #check the number of slices for this scan, and select the middle 28
    current_num_slice = img_data.shape[2]
    
    if current_num_slice > min_num_slice:
        #more than 28 slices, we need to select the middle one
        if current_num_slice%2 == 2:
            #this is an even number
            start_slice = int((current_num_slice - min_num_slice)/2)
            
        else:
            #this is an odd number
            start_slice = int((current_num_slice -1 - min_num_slice)/2)
        
        #in python, the end_slice is actually not included in the range
        end_slice = int(start_slice + min_num_slice)
    else:
        start_slice = 0
        end_slice = int(start_slice + min_num_slice)
     
    #grab the label for this scan
    current_label = labels['ICH']['CQ500-CT-'+str(ss)]
    current_scan_id = ss
    
    
    #now we generate data for slices in the range between start_slice and end_slice
    slice_index = 0 #this starts from 0, not the same as the original slice number in the original scan
    for j in range(start_slice, end_slice):
        #resize
        one_img_resize = resize(img_data[:, :, j], new_size)
        if normalize_img == 1:
            #normalize to 0~255
            img_processed = 255 * (one_img_resize-np.amin(one_img_resize))/(np.amax(one_img_resize) - np.amin(one_img_resize))      
        else:
            if three_channels == 1:
                #be careful! now each image will be 3D
                
                #first create an empty numpy array for the channeled ones
                img_processed=np.empty((new_size[0], new_size[1], len(channel_param)))
            
                for c in range(0, len(channel_param)):
                    img_processed[:, :, c] = create_channel(one_img_resize, channel_param[c][0], channel_param[c][1])
                    
            else:
                img_processed = one_img_resize
        #save this img_processed as a numpy array
        img_file_name = 'Slices/' + 'CQ500-CT-' + str(ss) + '_Slice' + str(slice_index) + '.npy'
        np.save(img_file_name, img_processed)
        
        #create a dictionary for label and scan_id and save it
        label_info = {'scan_id': current_scan_id, 'label': current_label}
        img_label_file_name = 'Labels/' + 'CQ500-CT-' + str(ss) + '_Label' + str(slice_index) + '.npy'
        np.save(img_label_file_name, label_info)
        
        print(label_info)
        
        slice_index = slice_index+1

NameError: name 'num_slice_df' is not defined

In [25]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
from skimage.transform import resize

# Configuration
nifti_folder = 'D:/CQ500/ConvOuch-master/ConvOuch-master/NiftiFiles'
min_num_slice = 28  # Adjust as necessary
new_size = (224, 224)  # Example size, adjust as necessary
normalize_img = 1
three_channels = 0  # Set to 1 if using three channels
channel_param = [(1, 0), (0.5, 0), (0, 0)]  # Example params if using three channels
label_file_name = 'summary_label.csv'  # Path to your label file

# Load the labels
labels = pd.read_csv(label_file_name)
labels.set_index('name', inplace=True)

# Ensure output directories exist
os.makedirs('Slices', exist_ok=True)
os.makedirs('Labels', exist_ok=True)

# Traverse through each scan directory
for i in range(1, 491):  # CQ500-CT-1 to CQ500-CT-490
    folder_name = f'CQ500-CT-{i}'
    folder_path = os.path.join(nifti_folder, folder_name)
    
    # Check if directory exists
    if os.path.isdir(folder_path):
        # Find the NIfTI file
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.nii.gz'):  # Ensure it's a NIfTI file
                scan_fn = os.path.join(folder_path, file_name)
                
                # Load the scan
                try:
                    img = nib.load(scan_fn)
                    img_data = img.get_fdata()
                    
                    # Check the number of slices for this scan
                    current_num_slice = img_data.shape[2]
                    
                    # Determine slice range to use
                    if current_num_slice > min_num_slice:
                        start_slice = (current_num_slice - min_num_slice) // 2
                        end_slice = start_slice + min_num_slice
                    else:
                        start_slice = 0
                        end_slice = current_num_slice
                    
                    # Get the label for this scan
                    scan_id = f'CQ500-CT-{i}'
                    if scan_id not in labels.index:
                        print(f"Label not found for {scan_id}")
                        continue
                    
                    current_label = labels.loc[scan_id, 'ICH']
                    
                    # Generate data for slices in the range between start_slice and end_slice
                    for slice_index, j in enumerate(range(start_slice, end_slice)):
                        # Resize the image
                        one_img_resize = resize(img_data[:, :, j], new_size)
                        
                        # Normalize the image if necessary
                        if normalize_img == 1:
                            img_processed = 255 * (one_img_resize - np.amin(one_img_resize)) / (np.amax(one_img_resize) - np.amin(one_img_resize))
                        else:
                            if three_channels == 1:
                                img_processed = np.empty((new_size[0], new_size[1], len(channel_param)))
                                for c in range(len(channel_param)):
                                    img_processed[:, :, c] = create_channel(one_img_resize, channel_param[c][0], channel_param[c][1])
                            else:
                                img_processed = one_img_resize
                        
                        # Save this processed image as a numpy array
                        img_file_name = f'Slices/{scan_id}_Slice{slice_index}.npy'
                        np.save(img_file_name, img_processed)
                        
                        # Create a dictionary for the label and scan_id and save it
                        label_info = {'scan_id': scan_id, 'label': current_label}
                        img_label_file_name = f'Labels/{scan_id}_Slice{slice_index}.npy'
                        np.save(img_label_file_name, label_info)
                        
                        # Print the label info
                        print(f"Saved slice {slice_index} for {scan_id}: {label_info}")
                
                except Exception as e:
                    print(f"Error processing {scan_fn}: {e}")
    else:
        print(f"Directory does not exist: {folder_path}")


Saved slice 0 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 1 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 2 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 3 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 4 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 5 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 6 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 7 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 8 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 9 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 10 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 11 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 12 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Saved slice 13 for CQ500-CT-1: {'scan_id': 'CQ500-CT-1', 'label': True}
Sa

Saved slice 23 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 24 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 25 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 26 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 27 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 0 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 1 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 2 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 3 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 4 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 5 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 6 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 7 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'label': False}
Saved slice 8 for CQ500-CT-3: {'scan_id': 'CQ500-CT-3', 'la

Saved slice 27 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 0 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 1 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 2 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 3 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 4 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 5 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 6 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 7 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 8 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 9 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 10 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 11 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'label': False}
Saved slice 12 for CQ500-CT-5: {'scan_id': 'CQ500-CT-5', 'lab

Saved slice 26 for CQ500-CT-9: {'scan_id': 'CQ500-CT-9', 'label': False}
Saved slice 27 for CQ500-CT-9: {'scan_id': 'CQ500-CT-9', 'label': False}
Saved slice 0 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 1 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 2 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 3 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 4 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 5 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 6 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 7 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 8 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 9 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 10 for CQ500-CT-10: {'scan_id': 'CQ500-CT-10', 'label': True}
Saved slice 11 for CQ500-CT-10: {'scan_id': 'CQ500

Saved slice 0 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 1 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 2 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 3 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 4 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 5 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 6 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 7 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 8 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 9 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 10 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 11 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 12 for CQ500-CT-12: {'scan_id': 'CQ500-CT-12', 'label': True}
Saved slice 13 for CQ500-CT-12: {'scan_id': 'CQ5

...

Saved slice 0 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 1 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 2 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 3 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 4 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 5 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 6 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 7 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 8 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 9 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 10 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 11 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 12 for CQ500-CT-480: {'scan_id': 'CQ500-CT-480', 'label': True}
Saved slice 13 for CQ5

Saved slice 0 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 1 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 2 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 3 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 4 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 5 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 6 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 7 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 8 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 9 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 10 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 11 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved slice 12 for CQ500-CT-483: {'scan_id': 'CQ500-CT-483', 'label': False}
Saved sli

Saved slice 24 for CQ500-CT-485: {'scan_id': 'CQ500-CT-485', 'label': True}
Saved slice 25 for CQ500-CT-485: {'scan_id': 'CQ500-CT-485', 'label': True}
Saved slice 26 for CQ500-CT-485: {'scan_id': 'CQ500-CT-485', 'label': True}
Saved slice 27 for CQ500-CT-485: {'scan_id': 'CQ500-CT-485', 'label': True}
Saved slice 0 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 1 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 2 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 3 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 4 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 5 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 6 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 7 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 8 for CQ500-CT-486: {'scan_id': 'CQ500-CT-486', 'label': False}
Saved slice 

Saved slice 1 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 2 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 3 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 4 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 5 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 6 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 7 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 8 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 9 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 10 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 11 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 12 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 13 for CQ500-CT-488: {'scan_id': 'CQ500-CT-488', 'label': True}
Saved slice 14 for CQ

Saved slice 5 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 6 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 7 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 8 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 9 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 10 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 11 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 12 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 13 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 14 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 15 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 16 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Saved slice 17 for CQ500-CT-490: {'scan_id': 'CQ500-CT-490', 'label': False}
Save

ignore below, they are from previous versions

In [ ]:
#we will define a function, given a numpy array of scan numbers, 
#it will prepare the image data and label for us
def prepare_image_and_label(scan_list, filename, label_file_name=label_file_name, nifti_folder=nifti_folder,
                           new_size=new_size, normalize_img=normalize_img, three_channels=three_channels):
    #input:
    #scan_list: a np array of scan numbers, e.g. range(0, 3)
    #file name: a string, will be the name of the saved numpy array for both images and labels
    #label_file_name, the name of the label file
    #nifti_folder: the folder of all of the nifti files
    #new_size: the size of the resized images
    #normalize_img: whether normalize to 0~1 or keep the original range
    
    #load in the label file
    labels = pd.read_csv(label_file_name)

    #use the scan name as indes
    labels.set_index('name', inplace=True)
    
    if 'img_input' in locals():
        del img_input

    if 'label_input' in locals():
        del label_input

    for ss in scan_list:
        #for each scan read in the nifti file
        i = ss[0]
        scan_fn = nifti_folder + '/CQ500-CT-' + str(i) + '/CQ500-CT-' + str(i) + '.nii.gz'
        img = nib.load(scan_fn)
        img_data = img.get_data()

        #make sure to clear the processed img from previous scans
        if 'scan_imgs' in locals():
            del scan_imgs
        
        if three_channels == 1:
            #4d
            scan_imgs=np.empty((new_size[0], new_size[1], len(channel_param), img_data.shape[-1]))
        else:
            #3d
            scan_imgs=np.empty((new_size[0], new_size[1], img_data.shape[-1]))
                

        #for each slice in the scan, resize it to the new size
        for j in range(0, img_data.shape[-1]):
            #resize
            one_img_resize = resize(img_data[:, :, j], new_size)

            if normalize_img == 1:
                #normalize to 0~255
                img_processed = 255 * (one_img_resize-np.amin(one_img_resize))/(np.amax(one_img_resize) - np.amin(one_img_resize))      
                scan_imgs[:,:,j] = img_processed
                
            else:
                if three_channels == 1:
                    #be careful! now each image will be 3D and we will be dealing with 4D data
                    #when they are stacked together
                
                    #first create an empty numpy array for the channeled ones
                    img_processed=np.empty((new_size[0], new_size[1], len(channel_param)))
            
                    for c in range(0, len(channel_param)):
                        img_processed[:, :, c] = create_channel(one_img_resize, channel_param[c][0], channel_param[c][1])
                        
                    scan_imgs[:,:,:,j] = img_processed
                else:
                    img_processed = one_img_resize
                    scan_imgs[:,:,j] = img_processed
            
        #also, pull out the corresponding label for each scan 
        #create a numpy array for all of the slices
        current_label = np.zeros((img_data.shape[-1], 1)) + labels['ICH']['CQ500-CT-'+str(i)]
        current_scan_id = np.zeros((img_data.shape[-1], 1)) + i
        
        #finally, stack the images from multiple scans into a single numpy array
        if 'img_input' in locals():
            img_input = np.concatenate((img_input, scan_imgs), axis=-1)
        else:
            img_input = scan_imgs.copy()

        if 'label_input' in locals():
            label_input = np.concatenate((label_input, current_label), axis=0)
        else:
            label_input = current_label.copy()
            
        if 'scan_id_input' in locals():
            scan_id_input = np.concatenate((scan_id_input, current_scan_id), axis=0)
        else:
            scan_id_input = current_scan_id.copy()
    
    
    #save the prepared image and label input with a certain name 'filename'
    np.save(filename + '_img', img_input)
    np.save(filename + '_label', label_input)
    np.save(filename + '_scan_id', scan_id_input)
    
    return img_input, label_input, scan_id_input

In [57]:
#we will define a function, given a numpy array of scan numbers, 
#it will prepare the image data and label for us
def prepare_image_and_label(scan_list, filename, label_file_name=label_file_name, nifti_folder=nifti_folder,
                           new_size=new_size, normalize_img=normalize_img, three_channels=three_channels):
    #input:
    #scan_list: a np array of scan numbers, e.g. range(0, 3)
    #file name: a string, will be the name of the saved numpy array for both images and labels
    #label_file_name, the name of the label file
    #nifti_folder: the folder of all of the nifti files
    #new_size: the size of the resized images
    #normalize_img: whether normalize to 0~1 or keep the original range
    
    #get a list of all of the nifti file names
    #we will need to change the names here!
    scan_fn_list = []
    for i in scan_list:
        scan_fn_list.append(nifti_folder + '/CQ500-CT-' + str(i[0]) + '/CQ500-CT-' + str(i[0]) + '_5mm.nii')

    #load in the label file
    labels = pd.read_csv(label_file_name)

    #use the scan name as indes
    labels.set_index('name', inplace=True)
    
    if 'img_input' in locals():
        del img_input

    if 'label_input' in locals():
        del label_input

    for i in range(0, (scan_list.shape[0])):
        #for each scan read in the nifti file
        scan_fn = scan_fn_list[i]
        img = nib.load(scan_fn)
        img_data = img.get_data()

        #make sure to clear the processed img from previous scans
        if 'scan_imgs' in locals():
            del scan_imgs
        
        if three_channels == 1:
            #4d
            scan_imgs=np.empty((new_size[0], new_size[1], len(channel_param), img_data.shape[-1]))
        else:
            #3d
            scan_imgs=np.empty((new_size[0], new_size[1], img_data.shape[-1]))
                

        #for each slice in the scan, resize it to the new size
        for j in range(0, img_data.shape[-1]):
            #resize
            one_img_resize = resize(img_data[:, :, j], new_size)

            if normalize_img == 1:
                #normalize to 0~255
                img_processed = 255 * (one_img_resize-np.amin(one_img_resize))/(np.amax(one_img_resize) - np.amin(one_img_resize))      
                scan_imgs[:,:,j] = img_processed
                
            else:
                if three_channels == 1:
                    #be careful! now each image will be 3D and we will be dealing with 4D data
                    #when they are stacked together
                
                    #first create an empty numpy array for the channeled ones
                    img_processed=np.empty((new_size[0], new_size[1], len(channel_param)))
            
                    for c in range(0, len(channel_param)):
                        img_processed[:, :, c] = create_channel(one_img_resize, channel_param[c][0], channel_param[c][1])
                        
                    scan_imgs[:,:,:,j] = img_processed
                else:
                    img_processed = one_img_resize
                    scan_imgs[:,:,j] = img_processed
            
        #also, pull out the corresponding label for each scan 
        #create a numpy array for all of the slices
        current_label = np.zeros((img_data.shape[-1], 1)) + labels['ICH']['CQ500-CT-'+str(i)]
        current_scan_id = 
        
        #finally, stack the images from multiple scans into a single numpy array
        if 'img_input' in locals():
            img_input = np.concatenate((img_input, scan_imgs), axis=-1)
        else:
            img_input = scan_imgs.copy()

        if 'label_input' in locals():
            label_input = np.concatenate((label_input, current_label), axis=0)
        else:
            label_input = current_label.copy()
    
    
    #save the prepared image and label input with a certain name 'filename'
    np.save(filename + '_img', img_input)
    np.save(filename + '_label', label_input)
    
    return img_input, label_input

In [84]:
train_index = np.array([range(1, 120)])
train_index = np.concatenate((train_index, np.array([range(121, 163)])), axis=1)
train_index = np.concatenate((train_index, np.array([range(164, 336)])), axis=1)
train_index = np.concatenate((train_index, np.array([range(337, 401)])), axis=1)

test_index = np.array([range(401, 490)])

train_index = train_index.T
test_index = test_index.T

#smartly divide the whole training set into severl subset
num_train = train_index.shape[0]
num_test = test_index.shape[0]

num_scan_per_set = 10

num_train_set = np.ceil(num_train/num_scan_per_set).astype(int)
num_test_set = np.ceil(num_test/num_scan_per_set).astype(int)

for i in range(1, num_train_set+1):
    print(i)
    if i < num_train_set:
        current_index = train_index[(i-1)*num_scan_per_set : i*num_scan_per_set]
    else:
        current_index = train_index[(i-1)*num_scan_per_set : num_train]
    
    img_input, label_input, scan_id_input = prepare_image_and_label(current_index, 'train'+str(i))

for i in range(1, num_test_set+1):
    print(i)
    if i < num_test_set:
        current_index = test_index[(i-1)*num_scan_per_set : i*num_scan_per_set]
    else:
        current_index = test_index[(i-1)*num_scan_per_set : num_test]
        
    img_input, label_input, scan_id_input = prepare_image_and_label(current_index, 'test'+str(i))
    
    
    


1


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


2


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


3


...

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


8


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


9


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [61]:
img_input, label_input = prepare_image_and_label(train_index, 'train1')

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [63]:
img_input = np.load('train1_img.npy')
print(img_input.shape)
print(label_input.shape)

(224, 224, 3, 1017)
(1017, 1)


The following are the original non-function code

In [114]:
#get a list of all of the nifti file names
#we will need to change the names here!
scan_list=range(0, 3)
scan_fn_list = []
for i in scan_list:
    scan_fn_list.append(nifti_folder + '/nifti_test' + str(i) + '.nii')
    
#load in the label file
labels = pd.read_csv(label_file_name)

#use the scan name as indes
labels.set_index('name', inplace=True)

In [115]:
if 'img_input' in locals():
    del img_input

if 'label_input' in locals():
    del label_input
    
for i in scan_list:
    #for each scan read in the nifti file
    scan_fn = scan_fn_list[i]
    img = nib.load(scan_fn)
    img_data = img.get_data()
    
    #create an empty numpy array for the processed images
    img_processed=np.empty((new_size[0], new_size[1], img_data.shape[-1]))
    
    #for each slice in the scan, resize it to the new size
    for i in range(0, img_data.shape[-1]):
        #resize
        one_img_resize = resize(img_data[:, :, i], new_size)
        
        if normalize_img == 1:
            #normalize to 0~1
            one_img_resize_norm = (one_img_resize-np.amin(one_img_resize))/(np.amax(one_img_resize) - np.amin(one_img_resize))

        
        #then stack back to the 36 images list
        img_processed[:,:,i] = one_img_resize
        
    
    #also, pull out the corresponding label for each scan 
    #create a numpy array for all of the slices
    current_label = np.zeros((img_data.shape[-1], 1)) + labels['ICH']['CQ500-CT-'+str(i)]
    
    #finally, stack the images from multiple scans into a single numpy array
    if 'img_input' in locals():
        img_input = np.dstack((img_input, img_processed))
    else:
        img_input = img_processed.copy()
        
    if 'label_input' in locals():
        label_input = np.concatenate((label_input, current_label), axis=0)
    else:
        label_input = current_label.copy()
        

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
